In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Bio import SeqIO
import csv

import tqdm
import pickle
from torch.utils.data import DataLoader

import proteinbert_gen.constants as consts
from proteinbert_gen.dataset import sprot_train

In [12]:
ofile = "../data/uniprot_sprot_1m.csv"

debug = False
count = 0
max_count = 1_000_000
min_len = 10
max_len = 510 # 512 - 2 (i.e. start + end)

def valid_aas(seq):
    for aa in seq:
        if aa not in consts.ALL_AAS_SET:
            return False
    return True
    
with open(ofile, "w", newline="") as fout, open("/data/protein-modeling/uniprot/uniprot_sprot.dat") as f:
    writer = csv.writer(fout)
    
    for record in SeqIO.parse(f, "swiss"):
        if len(record.seq) > max_len:
            continue
        if len(record.seq) < min_len:
            continue

        if not valid_aas(record.seq):
            continue

        count += 1

        if debug:
            print(record.id, record.seq[:32])
            # record.annotations["protein_existence"]; https://www.uniprot.org/help/protein_existence
        pid, seq = record.id, record.seq
        desc = record.annotations.get("comment", "<NO_DATA>")

        writer.writerow([count, pid, seq, desc])
        
        if count > max_count:
            break

In [13]:
# generate word freq dictionary
# word_freq = torch.zeros((consts.VOCAB_SIZE,), dtype=torch.int64)
word_freq = {token: 0 for token in consts.ALL_TOKENS}

for data in tqdm.tqdm(sprot_train):
    # print(data["seq"])
    for aa in data["seq"]:
        word_freq[aa] += 1

print(word_freq)

with open("../data/sprot_1m_word_freq_dict.pkl", "wb") as f:
    pickle.dump(word_freq, f)

  4%|██▋                                                                         | 11664/325340 [00:00<00:05, 61892.41it/s]


KeyError: 'B'

In [6]:
len(sprot_train)

325340